In [1]:
%matplotlib inline

import numpy as np
import edward as ed
import pandas as pd
import tensorflow as tf
from edward.models import Categorical, Normal

import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt

plt.style.use('ggplot')

/Users/sabber/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
## Sklearn related modules
from sklearn.utils import shuffle, class_weight
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

In [4]:
random_seed = 2018

In [5]:
ed.set_seed(random_seed)

In [6]:
# inspired by http://nipunbatra.github.io/2014/08/latexify/
plt.style.use(['default'])
params = {
    'text.latex.preamble': ['\\usepackage{gensymb}'],
    'image.origin': 'lower',
    'image.interpolation': 'nearest',
    'image.cmap': 'gray',
    'axes.grid': False,
    'savefig.dpi': 150,  # to adjust notebook inline plot size
    'axes.labelsize': 12, # fontsize for x and y labels (was 10)
    'axes.titlesize': 12,
    'font.size': 12, # was 10
    'legend.fontsize': 12, # was 10
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'text.usetex': True,
    'figure.facecolor':'white',
    'font.family': 'serif',
}
plt.rcParams.update(params)

### Import Data

In [7]:
## look data with pandas
train_file = "data/rupturemodel_train.txt"
val_file = "data/rupturemodel_xvalidate.txt"
test_file = "data/rupturemodel_test.txt"

df_train= pd.read_csv(train_file, sep=" ", header = None, dtype = np.float32)
df_val= pd.read_csv(val_file, sep=" ", header = None, dtype = np.float32)
df_test= pd.read_csv(test_file, sep=" ", header = None, dtype = np.float32)

features_name =  ['height', 'width', 'sxx', 'sxy', 'syy', 'sdrop', 'mud', 'dc', 'label']
df_train.columns = features_name
df_val.columns = features_name
df_test.columns = features_name

df_train_all = df_train.append(df_val)

### Data processing
We use StandardScaler preprocessing method of scikit. It standardizes features 
by removing the mean and scaling to unit variance

In [8]:
df = shuffle(df_train_all, random_state = random_seed)
df_train = shuffle(df_train, random_state = random_seed)
df_val = shuffle(df_val, random_state = random_seed)
df_test = shuffle(df_test, random_state = random_seed)


mms = StandardScaler()
train_scale = mms.fit(df.drop('label', axis=1))

xtrain = train_scale.transform(df.drop('label', axis=1))
ytrain =  df['label'].values

# xtrain = train_scale.transform(df_train.drop('label', axis=1))
# ytrain = df_train['label'].values

# xval = train_scale.transform(df_val.drop('label', axis=1))
# yval = df_val['label'].values

xtest = train_scale.transform(df_test.drop('label', axis=1))
ytest = df_test['label'].values

### Model prior weights

In [15]:
## Hyperparameters
D = len(features_name) - 1 # input size
l1 = 12 # Number of neurons in the first layer
BATCH_SIZE = 32
N = np.shape(xtrain)[0]

In [10]:
with tf.name_scope('model_prior'):
    
    W_0 = Normal(loc = tf.zeros([D, l1], name="weights/loc_w_0"), 
                 scale=tf.ones([D, l1], name="weights/scale_w_0"),
                 name  = 'weight_w_0')
    W_1 = Normal(loc=tf.zeros([l1, 2], name="weights/loc_w_1"), 
                 scale=tf.ones([l1, 2], name="weights/scale_w_1"),
                 name  = 'weight_w_1')
    b_0 = Normal(loc=tf.zeros(l1, name="weights/loc_b_0"), 
                 scale=tf.ones(l1, name="weights/scale_b_0"),
                 name  = 'weight_b_0')
    b_1 = Normal(loc=tf.zeros(2, name="weights/loc_b_1"),
                 scale=tf.ones(2, name="weights/scale_b_1"),
                 name  = 'weight_b_1')
    
#     x = np.array(xtrain, dtype = np.float32)
#     N = np.shape(x)[0]
    
    x_ = tf.placeholder(tf.float32, shape=(None, D))
    y_ = tf.placeholder(tf.int32, shape=(BATCH_SIZE))
    
    h = tf.nn.relu(tf.matmul(x_, W_0) + b_0)
    y = Categorical(tf.nn.sigmoid(tf.matmul(h, W_1) + b_1))

### Inference

In [11]:
with tf.name_scope('posterior'):
    qW_0 = Normal(loc=tf.get_variable("qW_0/loc", [D, l1]),
                  scale=tf.nn.softplus(tf.get_variable("qW_0/scale", [D, l1])))
    qW_1 = Normal(loc=tf.get_variable("qW_1/loc", [l1, 2]),
                  scale=tf.nn.softplus(tf.get_variable("qW_1/scale", [l1, 2])))
    qb_0 = Normal(loc=tf.get_variable("qb_0/loc", [l1]),
                  scale=tf.nn.softplus(tf.get_variable("qb_0/scale", [l1])))
    qb_1 = Normal(loc=tf.get_variable("qb_1/loc", [2]),
                  scale=tf.nn.softplus(tf.get_variable("qb_1/scale", [2])))

In [12]:
inference = ed.KLqp({W_0: qW_0, b_0: qb_0, W_1: qW_1, b_1: qb_1}, 
                    data = {y: y_})

# Initialse the infernce variables
inference.initialize(n_samples = 100, n_iter=5000, n_print=100, logdir='log/samples')

In [13]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [17]:
for _ in range(inference.n_iter):    
    perm = np.random.permutation(N)
    for i in range(0, N, BATCH_SIZE):
        batch_x = xtrain[perm[i:i+BATCH_SIZE]]
        batch_y = ytrain[perm[i:i+BATCH_SIZE]]
        info_dict = inference.update(feed_dict={x_: batch_x, y_: batch_y})
        inference.print_progress(info_dict)

FailedPreconditionError: Attempting to use uninitialized value qW_1/loc
	 [[Node: qW_1/loc/read = Identity[T=DT_FLOAT, _class=["loc:@qW_1/loc"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](qW_1/loc)]]

Caused by op 'qW_1/loc/read', defined at:
  File "/Users/sabber/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/sabber/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/sabber/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/sabber/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/sabber/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/Users/sabber/anaconda3/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 127, in start
    self.